<a href="https://colab.research.google.com/github/abxda/Descarga_Inegi_Censo_Denue_Duckdb/blob/main/Visualizaci%C3%B3n_Espacial_Interactiva_con_DuckDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import duckdb
import folium
from folium.plugins import MarkerCluster
from IPython.display import display

In [ ]:
# Usamos gdown para descargar el archivo .duckdb directamente a nuestro entorno.
print("\nPaso 2: Descargando la base de datos censo_denue.duckdb...")
# El ID del archivo de Google Drive
file_id = '1qzdEXMlh7Zctz_qxIHZ_eMPRExAlO2fF'
# El comando para descargar y renombrar el archivo
!gdown --id {file_id} -O censo_denue.duckdb
print("¡Base de datos descargada!")

In [ ]:
# Nos conectamos a la base de datos local y ejecutamos una consulta SQL.
# El objetivo es obtener la ubicación de todas las farmacias en el estado de
# Aguascalientes (clave de entidad '01') del DENUE.

In [ ]:
# Conectar a la base de datos DuckDB en modo de solo lectura.
con = duckdb.connect(database='censo_denue.duckdb', read_only=True)

# Definir la consulta SQL para obtener los datos que necesitamos.
# CORRECCIÓN: Se cambió 'nombre_actividad' por 'nombre_act' según el diccionario de datos.
query = """
SELECT
    latitud,
    longitud,
    nom_estab AS nombre_establecimiento
FROM denue
WHERE
    nombre_act LIKE '%Farmacias%' AND cve_ent = '01';
"""

# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas.
farmacias_df = con.execute(query).fetch_df()

# Es una buena práctica cerrar siempre la conexión a la base de datos.
con.close()

print(f"¡Extracción exitosa! Se encontraron {len(farmacias_df)} farmacias.")
print("Primeros 5 registros extraídos:")
print(farmacias_df.head())

In [ ]:
# Ahora, usamos Folium para visualizar los puntos. Folium es una capa sobre
# Leaflet.js que nos permite crear mapas desde Python.
#
# 1. Capa Base (Base Map): El "escenario". Usaremos OpenStreetMap.
# 2. Capa de Datos (Data Layer): Los "actores". Serán nuestras farmacias,
#    agrupadas en clústeres interactivos para evitar la sobredensidad.

if farmacias_df is not None and not farmacias_df.empty:
    print("\nPaso 5: Creando el mapa interactivo...")

    # Crear un objeto de mapa centrado en Aguascalientes.
    # location=[lat, lon], zoom_start define el acercamiento inicial.
    mapa_clusters = folium.Map(location=[21.8853, -102.2916], zoom_start=12, tiles="CartoDB positron")

    # Crear una capa de MarkerCluster. Esta capa manejará la lógica de agrupar
    # los puntos automáticamente según el nivel de zoom.
    marker_cluster = MarkerCluster().add_to(mapa_clusters)

    # Iterar sobre cada farmacia en nuestro DataFrame.
    for _, row in farmacias_df.iterrows():
        # Para cada farmacia, creamos un Marcador (un "pin").
        # El popup mostrará el nombre del establecimiento al hacer clic.
        folium.Marker(
            location=[row['latitud'], row['longitud']],
            popup=f"<strong>{row['nombre_establecimiento']}</strong>"
        ).add_to(marker_cluster) # ¡Importante! Añadimos el marcador al CLÚSTER, no al mapa directamente.

    print("¡Mapa generado! Mostrando el resultado a continuación...")
    # La última línea del código en una celda de Colab que devuelve un objeto
    # de Folium (como 'mapa_clusters') lo renderizará automáticamente.
    display(mapa_clusters)

else:
    print("\nNo se pudieron generar los mapas porque no se extrajeron datos.")